In [17]:
import cv2
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import hog
import matplotlib.pyplot as plt


In [18]:
# 1) load data
image_dir_1 = "../class_inconue"
image_dir_2 = "../logos"

images = []
labels = []

def read_data(images_dir , category ):
    global images,labels
    for filename in os.listdir(images_dir):
        if filename.endswith(".jpg"):
            img = cv2.imread(os.path.join(images_dir, filename))
            label = category  # label est le nom de la marque de la voiture
            images.append(img)
            labels.append(label)


read_data(image_dir_1 , 0 )
read_data(image_dir_2 , 1 )


In [19]:
# 2. Prétraitement des images
gray_images = []

for img in images:
    resized_img = cv2.resize(img, (128, 128))
    gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
    #new_vect_img = np.array(gray).flatten()
    gray_images.append(gray)

In [20]:
# 3. Extraction de fonctionnalités HOG
features = []
for img in gray_images:
    fd, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), visualize=True, channel_axis=None)
    features.append(fd)
    
len(features[0])

8100

In [21]:
# 4. Construction des vecteurs de caractéristiques
X = np.vstack(features)
y = labels

X.shape

(3851, 8100)

In [22]:
# 5. Entraînement du modèle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

classifier = SVC()

parameters = [{'kernel':['linear'],'gamma':[0.01, 0.001,0.002,0.003, 0.0001],'C':[0.001,0.01,0.1,1,10,100,1000]}]

grid_search = GridSearchCV(classifier,parameters)

grid_search.fit(X_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                          'gamma': [0.01, 0.001, 0.002, 0.003, 0.0001],
                          'kernel': ['linear']}])

In [24]:
# Affichage des meilleurs hyperparamètres et du score correspondant
print(f"Meilleurs hyperparamètres : {grid_search.best_params_}")
print(f"Score de validation croisée : {grid_search.best_score_}")

Meilleurs hyperparamètres : {'C': 0.01, 'gamma': 0.01, 'kernel': 'linear'}
Score de validation croisée : 0.9214285714285715


In [25]:
best_estimator = grid_search.best_estimator_

In [26]:
# 6. Évaluation du modèle
y_pred = best_estimator.predict(X_test)

print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95       536
           1       0.96      0.79      0.87       235

    accuracy                           0.93       771
   macro avg       0.94      0.89      0.91       771
weighted avg       0.93      0.93      0.92       771

Accuracy: 0.9260700389105059


In [27]:
import joblib

joblib.dump(best_estimator,"model_is_logo.joblib")

['model_is_logo.joblib']

In [28]:
from skimage import exposure

def is_logo(best_estimator , img_test_1 ):
    
    resized_img_test_1 = cv2.resize(img_test_1, (35, 35))
    gray_test_1 = cv2.cvtColor(resized_img_test_1, cv2.COLOR_BGR2GRAY)

    fd_test_1, hog_image = hog(gray_test_1, orientations=9, pixels_per_cell=(8, 8),
                            cells_per_block=(2, 2), visualize=True, channel_axis=None)

    fd_test_1 = fd_test_1.reshape(1,-1)

    y_pred_1 = best_estimator.predict(fd_test_1)
    
    if y_pred_1 == 1:
        
        bfilter = cv2.bilateralFilter(gray_test_1,5,17,17) # noise reduction 
        edged = cv2.Canny(bfilter,30,200) # Edge detection 
        
        plt.figure(figsize=(12,4))
        plt.subplot(131)
        plt.imshow(cv2.cvtColor(resized_img_test_1,cv2.COLOR_BGR2RGB))
        plt.subplot(132)
        hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
        plt.imshow(hog_image_rescaled,cmap=plt.cm.gray)
        plt.subplot(133)
        plt.imshow(cv2.cvtColor(edged,cv2.COLOR_BGR2RGB))
        
    return y_pred_1